# Poisson Processes via Kinetic Monte Carlo

This notebook builds homogeneous and inhomogeneous Poisson processes
directly from the kinetic Monte Carlo primitives:
- `next_time` for exponential waiting times
- `next_event` for event selection
- `step!` for updating algorithm time and steps

In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(path=joinpath(@__DIR__, ".."))
Pkg.instantiate()

using Random
using StatsBase
using MonteCarloX

## Homogeneous Poisson process (single channel)

A constant rate can be modeled as a single-channel KMC process.
The inter-arrival times are exponentially distributed with mean $1/\lambda$.

In [ ]:
rng = MersenneTwister(7)
alg = Gillespie(rng)

rate = 1.2
T = 10.0
arrival_times = Float64[]

while alg.time < T
    t_new, event = step!(alg, [rate])
    push!(arrival_times, t_new)
end

inter_arrival = diff([0.0; arrival_times])
println("Events: $(alg.steps)")
println("Mean inter-arrival: $(round(mean(inter_arrival), digits=3))")
println("Expected mean:      $(round(1 / rate, digits=3))")

## Inhomogeneous Poisson via thinning

We use a time-dependent rate vector and sample waiting times
from a dominating homogeneous Poisson process.
Accepted events are chosen using `next_event`.

In [ ]:
rng = MersenneTwister(21)
alg = Gillespie(rng)

rate_vec(t) = [0.6 + 0.3 * sin(t), 0.7 + 0.2 * cos(t)]
generation_rate = [1.0, 1.0]

function thin_step!(alg, rate_vec, generation_rate)
    t0 = alg.time
    total_generation = sum(generation_rate)
    dt = next_time(alg.rng, τ -> sum(rate_vec(t0 + τ)), total_generation)
    t_new = t0 + dt
    rates_now = rate_vec(t_new)
    event = next_event(alg.rng, rates_now)
    alg.time = t_new
    alg.steps += 1
    return t_new, event
end

T = 20.0
counts = zeros(Int, 2)
times = Float64[]

while alg.time < T
    t_new, event = thin_step!(alg, rate_vec, generation_rate)
    counts[event] += 1
    push!(times, t_new)
end

println("Events: $(alg.steps)")
println("Channel counts: $(counts)")
println("Final time: $(round(alg.time, digits=3))")